# 화학적 유사도 분석 (Chemical Similarity Analysis)

이 노트북은 학습 데이터(`Human_PK_data.csv`)와 외부 검증 데이터(`All_external_test_Sets.csv`) 간의 화학적 구조 유사도를 분석합니다.
목표는 학습 데이터와 유사한 외부 데이터만 선별(Filtering)하여 모델의 유효 검증 도메인을 설정하는 것입니다.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from tqdm import tqdm

# 시각화 설정
sns.set(style="whitegrid", font_scale=1.2)
plt.rcParams["figure.figsize"] = (10, 6)
try:
    plt.rcParams['font.family'] = 'AppleGothic'
except:
    pass

## 1. 데이터 로드

In [ ]:
# 경로 설정
PROJECT_ROOT = ".."
TRAIN_DATA_PATH = os.path.join(PROJECT_ROOT, "data", "Human_PK_data.csv")
EXTERNAL_DATA_PATH = os.path.join(PROJECT_ROOT, "External_Test_ALL_Lombardo_FDA_CL_fu_V2", "All_external_test_Sets.csv")

# 데이터 로드
df_train = pd.read_csv(TRAIN_DATA_PATH)
df_ext = pd.read_csv(EXTERNAL_DATA_PATH)

print(f"학습 데이터 크기: {len(df_train)}")
print(f"외부 데이터 크기: {len(df_ext)}")

In [ ]:
# 컬럼 확인
print("Train columns:", df_train.columns.tolist())
print("External columns:", df_ext.columns.tolist())

## 2. Morgan Fingerprint 계산 및 유사도 측정
SMILES 컬럼을 자동으로 찾아서 Fingerprint를 생성하고, 각 외부 데이터 샘플에 대해 학습 데이터 중 가장 유사한 물질과의 Tanimoto Similarity를 계산합니다.

In [ ]:
def get_mol(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return mol
        return None
    except:
        return None

def get_fp(mol):
    if mol is None:
        return None
    return AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)

# SMILES 컬럼 찾기 (우선순위: Drug, smiles_r, SMILES 등)
train_smiles_col = 'Structure' if 'Structure' in df_train.columns else 'SMILES'
ext_smiles_col = 'smiles_r'

# Fingerprint 미리 계산
print("Computing Train Fingerprints...")
train_mols = [get_mol(s) for s in df_train[train_smiles_col]]
train_fps = [get_fp(m) for m in train_mols]
train_fps = [fp for fp in train_fps if fp is not None] # 유효한 것만 유지

print("Computing External Fingerprints...")
ext_mols = [get_mol(s) for s in df_ext[ext_smiles_col]]
ext_fps = [get_fp(m) for m in ext_mols]

# 유사도 계산
max_similarities = []

print("Calculating Similarities...")
for i, ext_fp in enumerate(tqdm(ext_fps)):
    if ext_fp is None:
        max_similarities.append(0.0)
        continue
    
    # Bulk similarity calculation against all train FPs
    sims = DataStructs.BulkTanimotoSimilarity(ext_fp, train_fps)
    max_similarities.append(max(sims))

df_ext['max_similarity'] = max_similarities

## 3. 유사도 분포 시각화 및 필터링 임계값 설정

In [ ]:
# 분포 시각화
plt.figure(figsize=(10, 5))
sns.histplot(df_ext['max_similarity'], bins=50, kde=True)
plt.axvline(0.4, color='r', linestyle='--', label='Threshold 0.4')
plt.axvline(0.5, color='orange', linestyle='--', label='Threshold 0.5')
plt.axvline(0.6, color='g', linestyle='--', label='Threshold 0.6')
plt.title('Distribution of Max Tanimoto Similarity (External vs Train)')
plt.xlabel('Max Tanimoto Similarity')
plt.ylabel('Count')
plt.legend()
plt.show()

print(df_ext['max_similarity'].describe())

## 4. 데이터 필터링 및 저장
임계값(기본 0.5)을 적용하여 필터링된 데이터셋을 생성합니다.

In [ ]:
THRESHOLD = 0.5

df_filtered = df_ext[df_ext['max_similarity'] >= THRESHOLD].reset_index(drop=True)

print(f"Filtering with threshold {THRESHOLD}...")
print(f"Original: {len(df_ext)} -> Filtered: {len(df_filtered)}")
print(f"Retention Rate: {len(df_filtered)/len(df_ext)*100:.1f}%")

# 저장
SAVE_PATH = os.path.join(".", "external_test_filtered.csv")
df_filtered.to_csv(SAVE_PATH, index=False)
print(f"Filtered dataset saved to: {SAVE_PATH}")